In [ ]:
!gdown  18cij6foP2ilw2NtadINSnYP6UUbenXCf

Downloading...
From: https://drive.google.com/uc?id=18cij6foP2ilw2NtadINSnYP6UUbenXCf
To: /content/scc5966.zip
100% 18.4M/18.4M [00:00<00:00, 118MB/s] 


In [ ]:
!unzip scc5966.zip

Archive:  scc5966.zip
replace movie_reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: movie_reviews.csv       
  inflating: movies_data.csv         
  inflating: test_data.csv           
  inflating: train_data .csv         
  inflating: users_data.csv          


In [ ]:
import pandas as pd 

df_movie_reviews = pd.read_csv('movie_reviews.csv')

df_movies_data = pd.read_csv('movies_data.csv')

df_test_data = pd.read_csv('test_data.csv')

df_train_data = pd.read_csv('train_data .csv')

df_users_data = pd.read_csv('users_data.csv')

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
def SVD_GD(df, user_len, items_len ,factors, global_avg, iters, learning_rate, reg):

  b_u = pd.Series(np.zeros(user_len), index=df.user_id.unique())
  b_i = pd.Series(np.zeros(items_len), index=df.movie_id.unique())

  P = pd.DataFrame(np.zeros((user_len, factors)), index = df.user_id.unique())
  Q = pd.DataFrame(np.zeros((items_len, factors)), index = df.movie_id.unique())

  for iter in range(iters):
    for item,row in df.iterrows(): 
      user = row['user_id']
      item = row['movie_id']
      rating = row['rating']

      pred = global_avg + b_u.loc[user] + b_i.loc[item] + np.dot(P.loc[user],Q.loc[item])

      e_ui = rating - pred
      b_u.loc[user] = b_u.loc[user] + (learning_rate*e_ui)
      b_i.loc[item] = b_i.loc[item] + (learning_rate*e_ui)
      for factor in range(factors):
        temp_puf = P.loc[user][factor]
        P.loc[user][factor] = P.loc[user][factor] + (learning_rate * ((e_ui*Q.loc[item][factor]) - (reg*P.loc[user][factor])))
        Q.loc[item][factor] = Q.loc[item][factor] + (learning_rate * ((e_ui*temp_puf) - (reg*Q.loc[item][factor])))
        
  return b_u, b_i, P, Q     
  
def pred_generic(global_avg, b_u, b_i, P, Q, user, item):
  return global_avg + b_u[user] + b_i[item] + np.dot(P.loc[user],Q.loc[item])
  
def generate(parameters):
  l= []
  
  for item,row in tqdm(df_validation.iterrows()):
    user = row['user_id']
    item = row['movie_id']
    if user not in df_train_data['user_id'].unique() and item in df_train_data['movie_id'].unique():
      l.append(items_avg[item])
    elif user in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
      l.append(user_avg[user])
    elif user not in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
      l.append(global_avg)
    else:
      parameters.append(user)
      parameters.append(item)

      value = pred_generic(parameters[0],parameters[1],parameters[2],parameters[3],parameters[4], parameters[5], parameters[6])
      if value < 1:
        value = 1
      if value > 5:
        value = 5

      l.append(value)
  
  return l


df_user_item = pd.DataFrame(index = df_train_data['user_id'].unique(), columns=df_train_data['movie_id'].unique())

for movie in df_train_data['movie_id'].unique():
  temp = df_train_data[df_train_data.movie_id == movie]
  df_user_item.loc[temp.user_id, movie] = temp.rating.to_list()
  
items_avg = np.mean(df_user_item)
user_avg = np.mean(df_user_item.T)
global_avg = np.nanmean(df_user_item.values.tolist())

user_len = len(df_train_data.user_id.unique())
items_len = len(df_train_data.movie_id.unique())

In [ ]:
reg = 0.01

factors =
iters =
lr =
		
b_u,b_i,P,Q = SVD_GD(df_train_data, user_len, items_len, factors, global_avg, iters, lr, reg)

y_pred = generate([global_avg, b_u, b_i, P, Q])


df_test_data['rating'] = y_pred
df_rating = df_test_data[['id','rating']]
df_rating = df_rating.set_index('id')
name = 'FC_SVD_GD_' + str(factors) + '_' + str(iters) + '_' + str(lr) + '_' + str(reg) + '.csv'
df_rating.to_csv('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/results-kaggle/' +name)